
# Workflow Practice

In this notebook, you’ll practice connecting to a SQLite database, creating tables from CSV files using Pandas, and writing SQL queries to explore the data.

The dataset comes from the [Bike Store Sample Database](https://www.kaggle.com/datasets/dillonmyrick/bike-store-sample-database) by Dillon Myrick. It models a fictional bike retailer with multiple stores, products, customers, and staff. Each table connects to others using foreign keys such as `customer_id`, `store_id`, and `product_id`.

You’ll:
- Connect to a local SQLite database
- Create tables using `pandas.to_sql()`
- Write and test SQL queries using `pd.read_sql()`

All of your work will take place directly in this notebook. Each question prompt is written below as a Markdown cell, followed by an empty code cell for you to write your query.



## Step 1: Connect to the Database

Run the following cell to connect to (or create) a SQLite database called `bike_store.db`.  
If the file doesn’t exist yet, SQLite will automatically create it.


In [ ]:
import sqlite3
import pandas as pd
import glob
import os



In [2]:
connection = sqlite3.connect("bike_store.db")
connection


## Step 2: Create Tables from CSV Files

The `data/` folder contains one CSV file per table.  
Use `pandas.read_csv()` and `DataFrame.to_sql()` to load each file into your database.

You only need to do this once.  
After that, you’ll be able to run queries against your newly created tables.


In [19]:
# Example for one file
products = pd.read_csv("data/products.csv")
products.to_sql("products", connection, if_exists="replace", index=False)

321

In [20]:

csv_files = glob.glob('data/*.csv')

for file_path in csv_files:
    df = pd.read_csv(file_path)
    table_name = os.path.splitext(os.path.basename(file_path))[0]
    df.to_sql(table_name, connection, if_exists="replace", index=False)
    print(f"Loaded {table_name}")


Loaded brands
Loaded categories
Loaded customers
Loaded orders
Loaded order_items
Loaded products
Loaded staffs
Loaded stocks
Loaded stores


### Verify Your Tables

Run a query to make sure your tables were created successfully.

In [21]:

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)


,name
0,file_path
1,modified_brands.csv
2,modified_categories.csv
3,modified_customers.csv
4,modified_orders.csv
5,modified_order_items.csv
6,data\brands.csv
7,data\categories.csv
8,data\customers.csv
9,data\orders.csv


## Step 3: Test a Simple Query

Before starting the exercises, confirm your connection and tables are working by previewing the first few rows of the `customers` table.

In [22]:

pd.read_sql("SELECT discount from order_items", connection)


,discount
0,0.20
1,0.07
2,0.05
3,0.05
4,0.20
...,...
4717,0.07
4718,0.20
4719,0.20
4720,0.07


### Q1. List all customers and their cities.

Return the first name, last name, and city of each customer. Sort alphabetically by last name and then by first name.

In [24]:
# Your query here
query1 = "SELECT customer_id, city FROM customers;"
pd.read_sql(query1, connection)

,customer_id,city
0,1,Orchard Park
1,2,Campbell
2,3,Redondo Beach
3,4,Uniondale
4,5,Sacramento
...,...,...
1440,1441,Staten Island
1441,1442,Brooklyn
1442,1443,Central Islip
1443,1444,Canandaigua


### Q2. Show all products and their prices.

Display each product name along with its list price. Sort by price in descending order.

In [27]:
# Your query here
query2 = "SELECT product_id, list_price FROM products;"
pd.read_sql(query2, connection)

,product_id,list_price
0,1,379.99
1,2,749.99
2,3,999.99
3,4,2899.99
4,5,1320.99
...,...,...
316,317,1999.99
317,318,1999.99
318,319,2799.99
319,320,3799.99


### Q3. Find all customers from California.

Return first name, last name, city, and state for all customers whose state is 'CA'. Sort alphabetically by last name.

In [29]:
# Your query here
query3 = "SELECT first_name, last_name, city, state FROM customers WHERE state = 'CA';"
pd.read_sql(query3, connection)

,first_name,last_name,city,state
0,Kasha,Todd,Campbell,CA
1,Tameka,Fisher,Redondo Beach,CA
2,Charolette,Rice,Sacramento,CA
3,Corene,Wall,Atwater,CA
4,Jamaal,Albert,Torrance,CA
...,...,...,...,...
279,Carola,Johns,Santa Cruz,CA
280,Lorrie,Justice,Pomona,CA
281,Tayna,Wade,Bakersfield,CA
282,Lee,Dunn,San Jose,CA


### Q4. Count how many products are in each category.

Return the category name and the number of products in that category. Sort from the highest count to the lowest.

In [31]:
# Your query here
query4 = "SELECT cat_name, product_count FROM " \
"(SELECT c.category_name as cat_name, c.category_id, count(p.product_id) as product_count from categories c " \
"JOIN products p ON c.category_id = p.category_id GROUP BY c.category_name, c.category_id); \
"
pd.read_sql(query4, connection)

,cat_name,product_count
0,Children Bicycles,59
1,Comfort Bicycles,30
2,Cruisers Bicycles,78
3,Cyclocross Bicycles,10
4,Electric Bikes,24
5,Mountain Bikes,60
6,Road Bikes,60


### Q5. Find all orders placed in 2018.

List the order ID, order date, and customer ID for orders made during the year 2018. Sort by order date.

In [35]:
# Your query here
query5 = "SELECT order_id, order_date, customer_id FROM orders " \
"WHERE SUBSTR(order_date, 1, 4) = '2018';"

pd.read_sql(query5, connection)

,order_id,order_date,customer_id
0,1324,2018-01-01,862
1,1325,2018-01-01,68
2,1326,2018-01-01,567
3,1327,2018-01-02,1026
4,1328,2018-01-02,1083
...,...,...,...
287,1611,2018-09-06,6
288,1612,2018-10-21,3
289,1613,2018-11-18,1
290,1614,2018-11-28,135


### Q6. Show each order with its total number of items.

Join the `orders` and `order_items` tables. Group by order ID and return the number of items per order.

In [38]:
# Your query here
query6 = "SELECT order_ids, order_count FROM (" \
"SELECT o.order_id as order_ids, sum(oi.quantity) as order_count FROM orders o " \
"JOIN order_items oi ON o.order_id = oi.order_id " \
"GROUP BY o.order_id);"

pd.read_sql(query6, connection)

,order_ids,order_count
0,1,8
1,2,3
2,3,2
3,4,2
4,5,4
...,...,...
1610,1611,4
1611,1612,8
1612,1613,3
1613,1614,5


### Q7. List total revenue per store.

Revenue = quantity * list_price * (1 - discount). Join `orders`, `order_items`, and `stores`, group by store name, and return total revenue.

In [41]:
# Your query here
query7 = "SELECT store_nm, sum(round(total_rev)) FROM " \
"(SELECT s.store_name as store_nm, o.order_id, oi.item_id, sum(oi.quantity*oi.list_price*(1-oi.discount)) as total_rev " \
"FROM stores s " \
"JOIN orders o ON s.store_id = o.store_id " \
"JOIN order_items oi ON o.order_id = oi.order_id " \
"GROUP BY s.store_name, o.order_id, oi.item_id)" \
"GROUP BY store_nm;"

pd.read_sql(query7, connection)

,store_nm,sum(round(total_rev))
0,Baldwin Bikes,5215704.0
1,Rowlett Bikes,867531.0
2,Santa Cruz Bikes,1605811.0


### Q8. Find the top 5 customers who spent the most overall.

Join `customers`, `orders`, and `order_items`. Sum the total spending per customer and return the top five spenders.

In [47]:
# Your query here
query8 = "SELECT cust_id, sum(round(total_spend)) as total_cust_spend FROM " \
"(SELECT c.customer_id as cust_id, o.order_id, oi.item_id, sum(oi.quantity*oi.list_price*(1-oi.discount)) as total_spend " \
"FROM customers c " \
"JOIN orders o ON c.customer_id = o.customer_id " \
"JOIN order_items oi ON o.order_id = oi.order_id " \
"GROUP BY c.customer_id, o.order_id, oi.item_id) " \
"GROUP BY cust_id " \
"ORDER BY total_cust_spend DESC LIMIT 5;"

pd.read_sql(query8, connection)

,cust_id,total_cust_spend
0,94,34808.0
1,10,33634.0
2,75,32803.0
3,6,32675.0
4,16,31926.0


### Q9. Show the best-selling product in each category.

Join `products`, `order_items`, and `categories`. For each category, identify the product with the highest total quantity sold.

In [105]:
# Your query here
query9 = "SELECT cat_name, prod_name, ROUND(total_order_rev) as revenue, prod_rev_rank FROM " \
"(SELECT cat_name, prod_name, total_order_rev, " \
"RANK() OVER(PARTITION BY cat_name ORDER BY total_order_rev DESC) as prod_rev_rank FROM " \
"(SELECT c.category_name as cat_name, p.product_name as prod_name, SUM(oi.quantity*oi.list_price*(1-oi.discount)) as total_order_rev " \
"FROM categories c " \
"JOIN products p ON c.category_id = p.category_id " \
"JOIN order_items oi ON p.product_id = oi.product_id " \
"JOIN orders o ON oi.order_id = o.order_id " \
"GROUP BY p.product_name)) " \
"WHERE prod_rev_rank = 1;"

pd.read_sql(query9, connection)

,cat_name,prod_name,revenue,prod_rev_rank
0,Children Bicycles,Electra Cruiser 1 (24-Inch) - 2016,72071.0,1
1,Comfort Bicycles,Electra Townie Original 7D EQ - 2016,155169.0,1
2,Cruisers Bicycles,Electra Townie Original 7D EQ - Women's - 2016,66509.0,1
3,Cyclocross Bicycles,Surly Straggler 650b - 2016,226766.0,1
4,Electric Bikes,Trek Conduit+ - 2016,389249.0,1
5,Mountain Bikes,Trek Slash 8 27.5 - 2016,555559.0,1
6,Road Bikes,Trek Domane SLR 6 Disc - 2017,211585.0,1


### Q10. Identify the employees (staff) who processed the most orders.

Join `staffs` and `orders`. Count the number of orders handled by each staff member and return the results sorted by highest total.

In [107]:
# Your query here
query10 = "SELECT staff_name, sum(order_count) FROM " \
"(SELECT CONCAT(s.first_name, ' ', s.last_name) as staff_name, count(o.order_id) as order_count " \
"FROM staffs s JOIN orders o ON s.staff_id = o.staff_id " \
"GROUP BY staff_name)" \
"GROUP BY staff_name;"

pd.read_sql(query10, connection)

,staff_name,sum(order_count)
0,Genna Serrano,184
1,Kali Vargas,88
2,Layla Terrell,86
3,Marcelene Boyer,553
4,Mireya Copeland,164
5,Venita Daniel,540
